<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/Korean_CTM_%EB%AA%85%EC%82%AC%EC%82%AC%EC%9A%A9_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
# with open("discrimination.txt", "r", encoding='utf-8-sig') as f:
#     text_file = f.readlines()

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [ ]:
documents = [line.strip() for line in open("discrimination.txt", encoding="utf-8").readlines()]

In [ ]:
documents[:5]

['text',
 '남자들은 도대체 왜 그러는걸까?',
 '말하면서 침 개많이 튀길것 같아.',
 '머리가 멍청해서 못읽는거아니냐?',
 '"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다"']

In [ ]:
korean_stopwords_path = "./korean_stopwords.txt"

# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.nouns(sent)
        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(documents)

In [ ]:
print(train_bow_embeddings.shape)

(4436, 2812)


In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(vocab)

2812

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

# Combined TM 학습하기

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=8, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [88720/88720]	Train Loss: 20.539516645041104	Time: 0:00:01.404779: : 20it [00:28,  1.41s/it]


# 토픽들
학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.  


> get_topic_lists


해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [ ]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['여자', '남자', '결혼', '화장', '한남', '임신', '머리', '설거지', '시집', '게임'],
             1: ['여자', '남자', '인간', '결혼', '인물', '집단', '여성', '남성', '게이', '군대'],
             2: ['세상', '한남', '회사', '조심', '인생', '민이', '인정', '성공', '누가', '아줌마'],
             3: ['사람', '나라', '인간', '이슬람', '집단', '인물', '학교', '가난', '사회', '인종'],
             4: ['유행',
              '군침',
              '수입',
              '스턴트맨',
              '이사',
              '일본어',
              '가운데',
              '실제',
              '일지',
              '장땡'],
             5: ['벌기',
              '농구',
              '벨라루스',
              '수입',
              '군침',
              '동유럽',
              '실제',
              '혼수',
              '업적',
              '구림'],
             6: ['공부', '여경', '아이돌', '부모', '주제', '뭔가', '회사', '인정', '직업', '얘기'],
             7: ['장애',
              '중국',
              '조선족',
              '중국인',
              '동네',
              '흑인',
              '한국',
              '요즘',
        

- 0번 토픽은 성 차별로 모델링이 된 듯 하다.
- 1번도 마찬가지다.
- 7번은 국가 차별로 보인다.
- 나머지 경향성은 잘 모르겠다.

# 시각화

In [ ]:
%pip install pyLDAvis

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)
     

Sampling: [10/10]: : 10it [00:17,  1.79s/it]
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
